<h2><b> GAME ENVIRONMENT CODE & BASIC FUNCTIONS</b></h2>

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
from game import Game
from racing_env import RaceGameEnv
from PIL import Image
from io import BytesIO
import numpy as np

In [14]:
game = Game()

<h2><b> TENSORFLOW ENVIRONMENT CODE </b></h2>

In [15]:

game.resetGame()


ss taken in 0.17550182342529297 seconds


0


In [16]:
for i in range(1):

    img = game.takess()
    im = Image.fromarray(img, 'RGB')    
    imgplot = plt.imshow(im,cmap=plt.cm.binary)
    plt.show()

    im.save("screenshot.jpeg")
    game.getSpead()
    game.move('up')

game.resetGame()

In [17]:
env = RaceGameEnv()



TypeError: Can't instantiate abstract class RaceGameEnv with abstract methods _reset, _step, action_spec, observation_spec